In [ ]:
import torch

# 1 Define the Model

If you want to use outside of a notebook then save the bellow model def in a `.py` file

The in the main file `from filename.py import FaceDetector`

In [3]:
import torchvision
import torch
import torch.nn as nn

backbone = torchvision.models.resnet50(pretrained=True)
# we will not use the last 3 layer of the resnet
# so backbone outpput will be 14x14
backbone_features = torch.nn.Sequential(*(list(backbone.children())[:-3])) 

class SingleFaceRegressionHead(nn.Module):
    def __init__(self, in_channels, hidden_dim=256):
        super(SingleFaceRegressionHead, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, hidden_dim, kernel_size=3, padding=1)
        self.act1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(hidden_dim, 1, kernel_size=1)
        
        
        self.bbox_reg = nn.Linear(196, 4)
        
        self._initialize_weights()
    
    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.conv2(x)

        x = torch.flatten(x, start_dim=2)        
        bbox = self.bbox_reg(x) 
        
        return bbox
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, std=0.01)
                nn.init.constant_(m.bias, 0)

class FaceDetector(nn.Module):
    def __init__(self):
        super(FaceDetector, self).__init__()
        self.backbone = backbone_features
        
        self.regression_head = SingleFaceRegressionHead(1024,1024)

    def forward(self, x):
        features = self.backbone(x)
        bbox_pred = self.regression_head(features)
        
        return bbox_pred


/home/kuba/.virenv/base/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kuba/.virenv/base/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# 2 init the model

In [7]:
path_to_saved_model = '/home/kuba/projects/swvl/testposttrain/bestt01.pt'
model = FaceDetector()
model.load_state_dict(torch.load(path_to_saved_model, weights_only=True))


<All keys matched successfully>

NOW you can use the model !

# 3 NOTE
Before using the modle note two things:
- How input must be structured
    - The pre process 
    - The Batch Dimension

### Batch Dim

In [8]:
test = torch.load('/home/kuba/Documents/data/raw/me-img/tensor/clear_straighton.pt')
test.shape

torch.Size([3, 224, 224])

In [9]:
# it is missing the batch dim so we will have error
# we must add batch dim
test = test.unsqueeze(dim=0)
test.shape

torch.Size([1, 3, 224, 224])

### The pre process

In [ ]:
#inputs must be tensors and be 224x224
#its also recommender they are normalized to how the model was trained
from torchvision import transforms
from PIL import Image
import torch


f_img = f'/home/kuba/Documents/data/raw/me-img/raw/sun.jpeg'
name = 'sun'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224), antialias=True)  # Resize all images to same dimensions
])

transform_normal =  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

img = Image.open(f_img)
img_tensor = transform(img) #you must
img_tensor = transform_normal(img_tensor) #optional

torch.save(img_tensor, f'/home/path/to/go/{name}.pt')

# 4 Using the model

In [12]:
#this can be done as such 
with torch.no_grad():
    res = model(test)

res

tensor([[[0.4962, 0.3899, 0.4162, 0.5635]]])

The output is x(center), y(center), width, height